In [ ]:
from langchain_chroma import Chroma
import numpy as np

In [5]:
from datasets import load_dataset

ds = load_dataset("rajpurkar/squad")

In [ ]:
ds

从训练集随机筛选4k，测试机随机筛选1k

In [6]:
raw_train_dataset = ds["train"]
raw_test_dataset = ds["validation"]
train_dataset = raw_train_dataset.train_test_split(train_size=4000, seed=42)["train"]
test_dataset = raw_test_dataset.train_test_split(train_size=1000, seed=42)["train"]

In [7]:
test_dataset[0]

{'id': '57336755d058e614000b5a3c',
 'title': 'Warsaw',
 'context': 'The Saxon Garden, covering the area of 15.5 ha, was formally a royal garden. There are over 100 different species of trees and the avenues are a place to sit and relax. At the east end of the park, the Tomb of the Unknown Soldier is situated. In the 19th century the Krasiński Palace Garden was remodelled by Franciszek Szanior. Within the central area of the park one can still find old trees dating from that period: maidenhair tree, black walnut, Turkish hazel and Caucasian wingnut trees. With its benches, flower carpets, a pond with ducks on and a playground for kids, the Krasiński Palace Garden is a popular strolling destination for the Varsovians. The Monument of the Warsaw Ghetto Uprising is also situated here. The Łazienki Park covers the area of 76 ha. The unique character and history of the park is reflected in its landscape architecture (pavilions, sculptures, bridges, cascades, ponds) and vegetation (domestic a

In [8]:
len(test_dataset[0]['context'])

1643

build vector db

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
len(tokenizer(test_dataset[0]['context'])["input_ids"])

In [ ]:
!pip install -qU langchain-huggingface langchain-chroma sentence-transformers

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

c:\Users\1\.conda\envs\mdqa\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
embedding_bert = HuggingFaceEmbeddings(model_name="bert-base-uncased")

No sentence-transformers model found with name bert-base-uncased. Creating a new one with mean pooling.


In [ ]:
# from FlagEmbedding.inference.embedder.encoder_only.base import BaseEmbedder
# bert_embedding = BaseEmbedder("bert-base-uncased")

In [36]:
bert_embedding_hf = HuggingFaceEmbeddings(model_name="bert-base-uncased")

No sentence-transformers model found with name bert-base-uncased. Creating a new one with mean pooling.


在微调的时候，选择mean,而不是 cls


In [ ]:
bert_embedding_hf.embed_documents(["hello"])

[[-0.15500633418560028,
  0.09710049629211426,
  -0.020040372386574745,
  -0.27376317977905273,
  -0.23086081445217133,
  -0.2155546396970749,
  0.26284876465797424,
  0.10263242572546005,
  -0.1488824337720871,
  -0.2300889492034912,
  -0.2398369461297989,
  -0.14952145516872406,
  0.1169717088341713,
  0.06567886471748352,
  -0.3251933157444,
  -0.3183562457561493,
  -0.1009320393204689,
  -0.024247199296951294,
  0.019319968298077583,
  0.3070817291736603,
  -0.07263374328613281,
  -0.06883560866117477,
  0.0965522900223732,
  0.16381819546222687,
  0.1161859855055809,
  -0.04072198644280434,
  -0.19064342975616455,
  0.10296561568975449,
  -0.19433026015758514,
  -0.21518723666667938,
  -0.20153862237930298,
  -0.056326642632484436,
  0.08445053547620773,
  0.12889818847179413,
  -0.2662866413593292,
  -0.07339517772197723,
  0.06719475984573364,
  -0.059494003653526306,
  -0.49576306343078613,
  0.2632795572280884,
  -0.19147562980651855,
  -0.26232388615608215,
  -0.0078825652599

In [ ]:
def calc_recall(preds, truths, cutoffs):
    recalls = np.zeros(len(cutoffs))
    for text, truth in zip(preds, truths):
        for i, c in enumerate(cutoffs):
            recall = np.intersect1d(truth, text[:c])
            recall_num = min(len(recall), 1)
            recalls[i] += recall_num / max(min(c, len(truth)), 1)
            # recalls[i] += len(recall) / max(min(c, len(truth)), 1)
    recalls /= len(preds)
    return recalls

In [45]:
def eval_recall(embeddings, collection_name):
    vector_store = Chroma(
        collection_name=collection_name,
        embedding_function=embeddings,
        # persist_directory="./chroma_squad_1k_db",  # Where to save data locally, remove if not necessary
    )
    vector_store.add_texts(test_dataset['context'])
    res_texts = []
    truths = []

    for item in test_dataset:
        query = item["question"]
        truth = item["context"]
        tmp_res = [item.page_content for item in vector_store.similarity_search(query)]
        res_texts.append(tmp_res)
        truths.append([truth])
        cut_offs = [1, 2, 3, 4]
    recalls = calc_recall(res_texts, truths, cut_offs)
    for i, c in enumerate(cut_offs):
        print(f"recall@{c}: {recalls[i]}")

In [46]:
eval_recall(embeddings, "bge-m3")

recall@1: 0.808
recall@2: 0.887
recall@3: 0.923
recall@4: 0.946


In [48]:
eval_recall(bert_embedding_hf, "bert")

recall@1: 0.411
recall@2: 0.411
recall@3: 0.503
recall@4: 0.503


## 测试一下BERT模型